In [106]:
import pandas as pd
import os

import nltk
#nltk.download('stopwords')
from nltk.corpus import stopwords
words = set(stopwords.words('english'))

In [101]:
class DataFind(object):
    def __init__(self, folder, endswith, name):
        self.folder = folder
        self.endswith = endswith
        self.name = name
        
    def get_file_names(self):
        files = [i for i in os.listdir(self.folder) if i.endswith(self.endswith)]
        return files

    def choose_file(self):
        files = DataFind(self.folder,self.endswith, self.name).get_file_names()
        count = len(files)-1
        while count != -1:
            file_name = files[count]
            if self.name in file_name:
                return file_name
                break
            else:
                count -= 1
                continue
        else:
            print('File name that was input was not found in file names. Try again.')

    def get_data(self):
        file_name = DataFind(self.folder,self.endswith, self.name).choose_file()
        file_path = os.path.join(self.folder, file_name)
        headers = [f'{self.name}_DateCreated',f'{self.name}_ID',f'{self.name}_Tweet',f'{self.name}_Hashtag']
        df = pd.read_csv(file_path, names = headers, encoding = 'utf-8')
        return df
    
    def clean_data(self):
        df = DataFind(self.folder,self.endswith, self.name).get_data()
        for i in df.columns[1:-1]:
            df[i] = df[i].map(lambda x: str(x[2:].replace("\'","").strip()))
        
        df[f'{self.name}_Hashtag'] = [[i.replace("b\'","").replace("\'","").replace("]","").replace("[","").strip() for i in row.split(',')] for row in df[f'{self.name}_Hashtag']]
        
        return df
    

In [102]:
biden_frame = DataFind('Data','.csv','biden').clean_data()
trump_frame = DataFind('Data','.csv','trump').clean_data()
theright_frame = DataFind('Data','.csv','theright').clean_data()
theleft_frame = DataFind('Data','.csv','theleft').clean_data()

In [99]:
display(trump_frame)

,trump_DateCreated,trump_ID,trump_Tweet,trump_Hashtag
0,2020-10-05 18:45:06,1313188480579563521,OMFG there letting the Clown back out of #Walt...,"[walterreed, clownshow, trumpcovid19, trump]"
1,2020-10-05 18:45:05,1313188478587461632,@DrDenaGrayson @realDonaldTrump There are two ...,"[covid19, trump, trumpcovid19]"
2,2020-10-05 18:45:02,1313188463961935872,There was a shift over the weekend the media h...,[trump]
3,2020-10-05 18:45:00,1313188455787098112,#Trump will be leaving #WalterReed #today at 6...,"[trump, walterreed, today, breakingnews, docto..."
4,2020-10-05 18:44:58,1313188447826444303,#Trump Fuck this Trump Clan is making me \xf0\...,[trump]
...,...,...,...,...
73,2020-10-05 18:42:59,1313187948381245442,it has been all indeed a publicity stunt \xf0\...,"[trump, trumpfakecovid, trumpcovidhoax]"
74,2020-10-05 18:42:54,1313187928408031233,@TVKev I still cannot work out why the British...,[trump]
75,2020-10-05 18:42:52,1313187917825794053,"How in gods name are they allowing #Trump, sti...","[trump, covid19]"
76,2020-10-05 18:42:51,1313187915040583685,@realDonaldTrump This tweet is an insult to th...,[]
